# Introduction to FermiLib alpha version 0.1
### Ryan Babbush, Jarrod McClean, Ian Kivlichan, Damian Steiger, Thomas Haner, Vojtech Havlicek and Wei Sun
Note that all the examples below must be run sequentially within a section.

## Initializing the FermionOperator data structure

Fermionic systems are often treated in second quantization where arbitrary operators can be expressed using the fermionic creation and annihilation operators, $a^\dagger_k$ and $a_k$.  The fermionic ladder operators play a similar role to their qubit ladder operator counterparts, $\sigma^+_k$ and $\sigma^-_k$ but are distinguished by the cannonical fermionic anticommutation relations, $\{a^\dagger_i, a^\dagger_j\} = \{a_i, a_j\} = 0$ and $\{a_i, a_j^\dagger\} = \delta_{ij}$. Any weighted sums of products of these operators are represented with the FermionOperator data structure in FermiLib. The following are examples of valid FermionOperators:
\begin{align}
& a_1 \\
& 1.7 a^\dagger_3\\
&-1.7 \, a^\dagger_3 a_1\\
&(1 + 2i) \, a^\dagger_4 a^\dagger_3 a_9 a_1\\
&(1 + 2i) \, a^\dagger_4 a^\dagger_3 a_9 a_1 - 1.7 \, a^\dagger_3 a_1
\end{align}

The FermionOperator class is contained in $\textrm{ops/_fermion_operators.py}$. In order to support fast addition of FermionOperator instances, the class is implemented as hash table (python dictionary). The keys of the dictionary encode the strings of ladder operators and values of the dictionary store the coefficients. The strings of ladder operators are encoded as a tuple of 2-tuples which we refer to as the "terms tuple". Each ladder operator is represented by a 2-tuple. The first element of the 2-tuple is an int indictating the tensor factor on which the ladder operator acts. The second element of the 2-tuple is Boole: 1 represents raising and 0 represents lowering. For instance, $a^\dagger_8$ is represented in a 2-tuple as $(8, 1)$. Note that indices start at 0 and the identity operator is an empty list. Below we give some examples of operators and their terms tuple:
\begin{align}
I & \mapsto ()\\
a_1 & \mapsto ((1, 0),)\\
a^\dagger_3 & \mapsto ((3, 1),)\\
a^\dagger_3 a_1 & \mapsto ((3, 1), (1, 0))\\
a^\dagger_4 a^\dagger_3 a_9 a_1 & \mapsto ((4, 1), (3, 1), (9, 0), (1, 0))
\end{align}
Note that when initializing a single ladder operator one should be careful to add the comma after the inner pair. This is because in python ((1, 2)) = (1, 2) whereas ((1, 2),) = ((1, 2),). The "terms tuple" is usually convenient when one wishes to initialize a term as part of a coded routine. However, the terms tuple is not particularly intuitive. Accordingly, FermiLib also supports another user-friendly, string notation below. This representation is rendered when calling "print" on a FermionOperator.
\begin{align}
I & \mapsto ""\\
a_1 & \mapsto "1"\\
a^\dagger_3 & \mapsto "3\textrm{^}"\\
a^\dagger_3 a_1 & \mapsto "3\textrm{^} \,\, 1"\\
a^\dagger_4 a^\dagger_3 a_9 a_1 & \mapsto "4\textrm{^}\,\, 3\textrm{^} \,\, 9 \,\,1"
\end{align}
Let's initialize our first term! We do it two different ways below.

In [1]:
from fermilib.ops import FermionOperator

my_term = FermionOperator(((3, 1), (1, 0)))
print(my_term)

my_term = FermionOperator('3^ 1')
print(my_term)

1.0 [3^ 1]
1.0 [3^ 1]


The preferred way to specify the coefficient in FermiLib is to provide an optional coefficient argument. If not provided, the coefficient defaults to 1. The first method is preferred. The multiplication in the last method actually creates a copy of the term, which introduces some additional cost. All inplace operands (such as +=) modify classes where as operands such as + create copies. Important caveats are that the empty tuple FermionOperator(()) or the empty string FermionOperator('') initializes identity. Whereas an empty initializer FermionOperator() initializes the zero operator. We demonstrate some of these things below.

In [2]:
good_way_to_initialize = FermionOperator('3^ 1', -1.7)
print(good_way_to_initialize)

bad_way_to_initialize = -1.7 * FermionOperator('3^ 1')
print(bad_way_to_initialize)

identity = FermionOperator('')
print(identity)

zero_operator = FermionOperator()
print(zero_operator)

-1.7 [3^ 1]
-1.7 [3^ 1]
1.0 []
0


FermionOperator has only one attribute: .terms. This attribute is the dictionary which stores the term tuples.

In [3]:
my_operator = FermionOperator('4^ 1^ 3 9', 1. + 2.j)
print(my_operator)
print(my_operator.terms)

(1+2j) [4^ 1^ 3 9]
{((4, 1), (1, 1), (3, 0), (9, 0)): (1+2j)}


## Manipulating the FermionOperator data structure
So far we have explained how to initialize a single FermionOperator such as $-1.7 \, a^\dagger_3 a_1$. However, in general we will want to represent sums of these operators such as $(1 + 2i) \, a^\dagger_4 a^\dagger_3 a_9 a_1 - 1.7 \, a^\dagger_3 a_1$. To do this, just add together two FermionOperators! We demonstrate below.

In [4]:
from fermilib.ops import FermionOperator

term_1 = FermionOperator('4^ 3^ 9 1', 1. + 2.j)
term_2 = FermionOperator('3^ 1', -1.7)
my_operator = term_1 + term_2
print(my_operator)

my_operator = FermionOperator('4^ 3^ 9 1', 1. + 2.j)
term_2 = FermionOperator('3^ 1', -1.7)
my_operator += term_2
print('')
print(my_operator)

(1+2j) [4^ 3^ 9 1] +
-1.7 [3^ 1]

(1+2j) [4^ 3^ 9 1] +
-1.7 [3^ 1]


The print function prints each term in the operator on a different line. Note that the line my_operator = term_1 + term_2 creates a new object, which involves a copy of term_1 and term_2. The second block of code uses the inplace method +=, which is more efficient. This is especially important when trying to construct a very large FermionOperator. FermionOperators also support a wide range of builtins including, str(), repr(), *=, *, /, /=, +, +=, -, -=, - and **. Note that instead of supporting != and ==, we have the method .isclose(), since FermionOperators involve floats. We demonstrate some of these methods below.

In [5]:
term_1 = FermionOperator('4^ 3^ 9 1', 1. + 2.j)
term_2 = FermionOperator('3^ 1', -1.7)

my_operator = term_1 - 33. * term_2
print(my_operator)

my_operator *= 3.17 * (term_2 + term_1) ** 2
print('')
print(my_operator)

print('')
print(term_2 ** 3)

print('')
print(term_1.isclose(2.*term_1 - term_1))
print(term_1.isclose(my_operator))

(1+2j) [4^ 3^ 9 1] +
56.1 [3^ 1]

(9.161299999999999+18.322599999999998j) [4^ 3^ 9 1 3^ 1 3^ 1] +
(16.166999999999998-21.555999999999997j) [4^ 3^ 9 1 3^ 1 4^ 3^ 9 1] +
(16.166999999999998-21.555999999999997j) [4^ 3^ 9 1 4^ 3^ 9 1 3^ 1] +
(-34.87-6.34j) [4^ 3^ 9 1 4^ 3^ 9 1 4^ 3^ 9 1] +
513.9489299999999 [3^ 1 3^ 1 3^ 1] +
(-302.32289999999995-604.6457999999999j) [3^ 1 3^ 1 4^ 3^ 9 1] +
(-302.32289999999995-604.6457999999999j) [3^ 1 4^ 3^ 9 1 3^ 1] +
(-533.511+711.348j) [3^ 1 4^ 3^ 9 1 4^ 3^ 9 1]

-4.912999999999999 [3^ 1 3^ 1 3^ 1]

True
False


Additionally, there are a variety of methods that act on the FermionOperator data structure. Many of these methods are found in fermilib.transforms and serve to map the FermionOperator data structure to a different data structure (for instance, mapping to qubits). There are also a variety of helpful functions in fermilib.utils. We demonstrate a few here.

In [6]:
from fermilib.ops import hermitian_conjugated, normal_ordered
from fermilib.utils import eigenspectrum, commutator

term_1 = FermionOperator('4^ 3 3^', 1. + 2.j)
print(term_1)
print(hermitian_conjugated(term_1))
print(term_1.is_normal_ordered())

print('')
term_2 = normal_ordered(term_1)
print(term_2)
print(term_2.is_normal_ordered())

print('')
print(eigenspectrum(term_1 + hermitian_conjugated(term_1)))

print('')
print(commutator(term_1, term_2))

(1+2j) [4^ 3 3^]
(1-2j) [3 3^ 4]
False

(1+2j) [4^] +
(-1-2j) [4^ 3^ 3]
True

[-2.23606798 -2.23606798 -2.23606798 -2.23606798 -2.23606798 -2.23606798
 -2.23606798 -2.23606798  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          2.23606798  2.23606798
  2.23606798  2.23606798  2.23606798  2.23606798  2.23606798  2.23606798]

(-3+4j) [4^ 3 3^ 4^] +
(3-4j) [4^ 3 3^ 4^ 3^ 3] +
(3-4j) [4^ 4^ 3 3^] +
(-3+4j) [4^ 3^ 3 4^ 3 3^]


## The QubitOperator data structure
The QubitOperator data structure is another essential part of FermiLib. While the QubitOperator was originally developed for FermiLib, it is now part of the core ProjectQ library so that it can be interpreted by the ProjectQ compiler using the TimeEvolution gate. As the name suggests, QubitOperator is used to store qubit operators in almost exactly the same way that FermionOperator is used to store fermion operators. For instance $X_0 Z_3 Y_4$ is a QubitOperator. The internal representation of this as a terms tuple would be $((0, X), (3, Z), (4, Y))$. Note that one important difference between QubitOperator and FermionOperator is that the terms in QubitOperator are always sorted in order of tensor factor. We initialize some QubitOperators below.

In [8]:
from projectq.ops import QubitOperator

my_first_qubit_operator = QubitOperator('X1 Y2 Z3')
print(my_first_qubit_operator)
print(my_first_qubit_operator.terms)

operator_2 = QubitOperator('X3 Z4', 3.17)
operator_2 -= 77. * my_first_qubit_operator
print('')
print(operator_2)

1.0 X1 Y2 Z3
{((1, 'X'), (2, 'Y'), (3, 'Z')): 1.0}

3.17 X3 Z4 +
-77.0 X1 Y2 Z3


## Jordan-Wigner and Bravyi-Kitaev
FermiLib provides functions for mapping FermionOperators to QubitOperators.

In [9]:
from fermilib.ops import FermionOperator, hermitian_conjugated
from fermilib.transforms import jordan_wigner, bravyi_kitaev
from fermilib.utils import eigenspectrum
from projectq.ops import QubitOperator

fermion_operator = FermionOperator('2^ 0', 3.17)
fermion_operator += hermitian_conjugated(fermion_operator)
print(fermion_operator)

jw_operator = jordan_wigner(fermion_operator)
print('')
print(jw_operator)

bk_operator = bravyi_kitaev(fermion_operator)
print('')
print(bk_operator)

jw_spectrum = eigenspectrum(jw_operator)
bk_spectrum = eigenspectrum(bk_operator)
print('')
print(jw_spectrum)
print(bk_spectrum)

3.17 [2^ 0] +
3.17 [0^ 2]

(1.585+0j) X0 Z1 X2 +
(1.585+0j) Y0 Z1 Y2

(-1.585+0j) Y0 Y1 +
(-1.585+0j) X0 X1 Z2

[-3.17 -3.17  0.    0.    0.    0.    3.17  3.17]
[-3.17 -3.17  0.    0.    0.    0.    3.17  3.17]


We can also apply the Jordan-Wigner transform in reverse to map arbitrary QubitOperators to FermionOperators. Note that we also demonstrate the .compress() method (a method on both FermionOperators and QubitOperators) which removes zero entries.

In [10]:
from fermilib.transforms import reverse_jordan_wigner

my_operator = QubitOperator('X0 Y1 Z2', 88.)
my_operator += QubitOperator('Z1 Z4', 3.17)
print(my_operator)

mapped_operator = reverse_jordan_wigner(my_operator)
print('')
print(mapped_operator)

back_to_normal = jordan_wigner(mapped_operator)
back_to_normal.compress()
print('')
print(back_to_normal)

88.0 X0 Y1 Z2 +
3.17 Z1 Z4

-88j [1^ 0^] +
88j [1^ 0] +
88j [1 0^] +
-88j [1 0] +
176j [2^ 2 1^ 0^] +
-176j [2^ 2 1^ 0] +
-176j [2^ 2 1 0^] +
176j [2^ 2 1 0] +
3.17 [] +
-6.34 [1^ 1] +
-6.34 [4^ 4] +
12.68 [4^ 4 1^ 1]

88.0 X0 Y1 Z2 +
3.17 Z1 Z4


## The Hubbard model

FermiLib supports generating various models of fermions such as the Hubbard model. There are modules for generating Hamiltonians in fermilib.utils.

In [12]:
from fermilib.utils import fermi_hubbard
x_dimension = 2
y_dimension = 2
tunneling = 2.
coulomb = 1.
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 1
hubbard_model = fermi_hubbard(
    x_dimension, y_dimension, tunneling, coulomb, chemical_potential, magnetic_field, periodic, spinless)
print(hubbard_model)

jw_hamiltonian = jordan_wigner(hubbard_model)
print('')
print(jw_hamiltonian)

0.0 [] +
-0.25 [0^ 0] +
1.0 [0^ 0 1^ 1] +
-2.0 [0^ 1] +
-2.0 [1^ 0] +
1.0 [0^ 0 2^ 2] +
-2.0 [0^ 2] +
-2.0 [2^ 0] +
-0.25 [1^ 1] +
1.0 [1^ 1 3^ 3] +
-2.0 [1^ 3] +
-2.0 [3^ 1] +
0.25 [2^ 2] +
1.0 [2^ 2 3^ 3] +
-2.0 [2^ 3] +
-2.0 [3^ 2] +
0.25 [3^ 3]

(1+0j) I +
(-0.375+0j) Z0 +
(-0.375+0j) Z1 +
(0.25-0j) Z0 Z1 +
(-1+0j) Y0 Y1 +
(-1+0j) X0 X1 +
(-0.625+0j) Z2 +
(0.25-0j) Z0 Z2 +
(-1+0j) Y0 Z1 Y2 +
(-1+0j) X0 Z1 X2 +
(-0.625+0j) Z3 +
(0.25-0j) Z1 Z3 +
(-1+0j) Y1 Z2 Y3 +
(-1+0j) X1 Z2 X3 +
(0.25-0j) Z2 Z3 +
(-1+0j) Y2 Y3 +
(-1+0j) X2 X3


## Hamiltonians in the plane wave basis
FermiLib uses a third-party electronic structure package to compute molecular orbitals, Hamiltonians, energies, reduced density matrices, coupled cluster amplitudes, etc using Gaussian basis sets. However, this third-party electronic structure package has a restrictive GPL license. Accordingly, we cannot even mention it by name in this tutorial. While we provide scripts which interface between that package and FermiLib, we cannot discuss it here.

When using simpler basis sets such as plane waves, these packages are not needed. FermiLib comes with code which computes Hamiltonians in the plane wave basis. Note that when using plane waves, one is working with the periodized Coulomb operator, best suited for condensed phase calculations such as studying the electronic structure of a solid. To obtain these Hamiltonians one must choose to study the system without a spin degree of freedom (spinless), one must the specify dimension in which the calculation is performed (n_dimensions, usually 3), one must specify how many plane waves are in each dimension (grid_length) and one must specify the length scale of the plane wave harmonics in each dimension (length_scale) and also the locations and charges of the nuclei. One can generate these models with function plane_wave_hamiltonian() found in fermilib.utils. For simplicity, below we compute the Hamiltonian in the case of zero external charge (corresponding to the uniform electron gas, aka jellium). We also demonstrate that one can transform the plane wave Hamiltonian using a Fourier transform without effecting the spectrum of the operator.

In [37]:
from fermilib.utils import fourier_transform, jellium_model
from fermilib.transforms import jordan_wigner

n_dimensions = 1
grid_length = 3
length_scale = 1.
spinless = True

momentum_hamiltonian = jellium_model(n_dimensions, grid_length, length_scale, spinless)
momentum_qubit_operator = jordan_wigner(momentum_hamiltonian)
position_qubit_operator.compress()
print(momentum_qubit_operator)

position_hamiltonian = fourier_transform(momentum_hamiltonian, n_dimensions, grid_length, length_scale, spinless)
position_qubit_operator = jordan_wigner(position_hamiltonian)
position_qubit_operator.compress()
print('')
print (position_qubit_operator)

spectral_difference = eigenspectrum(momentum_qubit_operator) -  eigenspectrum(position_qubit_operator)
print('')
print(spectral_difference)

(19.50047638754088+0j) I +
(-9.71044945799746+0j) Z0 +
(0.15915494309189535+0j) Z1 +
(-9.71044945799746+0j) Z2 +
(-0.07957747154594767+0j) Z0 Z1 +
(-0.07957747154594767+0j) Z1 Z2 +
(-0.07957747154594767+0j) Z0 Z2

19.500476387540854 I +
-6.420581324301009 Z0 +
-3.289868133696451 Y0 Y1 +
-3.289868133696451 X0 X1 +
-3.289868133696454 Y0 Z1 Y2 +
-3.289868133696454 X0 Z1 X2 +
-6.4205813243010095 Z1 +
-3.289868133696451 Y1 Y2 +
-3.289868133696451 X1 X2 +
-6.420581324301009 Z2 +
-0.07957747154594766 Z0 Z1 +
-0.07957747154594763 Z0 Z2 +
-0.07957747154594766 Z1 Z2

[  2.84217094e-14   2.84217094e-14   2.13162821e-14   1.06581410e-14
   3.19744231e-14   2.13162821e-14   1.42108547e-14   2.13162821e-14]


## Basics of MolecularData class and making a chemical series

Perhaps the most useful features in FermiLib concern its interaction with open source electronic structure packages. Once again, we provide scripts to interact with one such package but cannot legally refer to it by name here.

Data from electronic structure calculations is generated using scripts which perform the calculations and then populate a FermiLib data structure called MolecularData. Very often, one would like to analyze a chemical series or look at many different Hamiltonians and sometimes the electronic structure calculations are either expensive to compute or difficult to converge (e.g. one needs to mess around with different types of SCF routines to make things converge). Accordingly, we anticipate that users will want some way to automatically database the results of their electronic structure calculations so that important data (such as the SCF intergrals) can be looked up on-the-fly if the user has computed them in the past. FermiLib supports a data provenance strategy which saves key results of the electronic structure calculation (including pointers to files containing large amounts of data, such as the molecular integrals) in an HDF5 container.

The MolecularData class stores information about molecules. One initializes a MolecularData object by specifying key parameters of a molecule such as its geometry, basis, multiplicity, charge and an optional string describing it. One can also initialize MolecularData simply by providing a string giving a filename where a previous MolecularData object was saved. One can save a MolecularData instance by calling the class's .save() method. This automatically saves the instance in a data folder specified during FermiLib installation. The name of the file is generated automatically from the instance attributes and optionally provided description. Alternatively, a filename can also be provided as an optional input if one wishes to manually name the file.

When electronic structure calculations are run using our scripts, the data files for the molecule are automatically updated. For instance, once one runs an FCI calculation on the molecule, the attribute MolecularData.fci_energy will be saved and set equal the FCI energy. If one wishes to later use that data they either initialize MolecularData with the instance filename or initialize the instance and then later call the .load() method.

Basis functions are based to initialization using a string such as "6-31g". Geometries can be specified using a simple txt input file (see geometry_from_file function in molecular_data.py) or can be passed using a simple python list format demonstrated below. Atoms are specified using a string for their atomic symbol. Distances should be provided in atomic units (Bohr). Below we initialize a simple instance of MolecularData without performing any electronic structure calculations.

We also demonstrate some functionality is the simple helper module, chemical_series.py. A chemical series is a progression of larger and larger molecules. Often, it is interesting to study chemical series because one can extrapolate the performance a method to a regime that is difficult to simulate. We demonstrate how two functions from chemical_series.py can be used make hydrogen rings or single atoms with the current spin multiplicity.

In [41]:
from fermilib.utils import MolecularData

# Set parameters to make a simple molecule.
diatomic_bond_length = .7414
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

# Make molecule and print out a few interesting facts about it.
molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)
print('Molecule has automatically generated name {}'.format(
    molecule.name))
print('Information about this molecule would be saved at:\n{}\n'.format(
    molecule.filename))
print('This molecule has {} atoms and {} electrons.'.format(
    molecule.n_atoms, molecule.n_electrons))
for atom, atomic_number in zip(molecule.atoms, molecule.protons):
    print('Contains {} atom, which has {} protons.'.format(
        atom, atomic_number))

Molecule has automatically generated name H2_sto-3g_singlet_0.7414
Information about this molecule would be saved at:
/usr/local/google/home/babbush/Desktop/fermilib/data/H2_sto-3g_singlet_0.7414

This molecule has 2 atoms and 2 electrons.
Contains H atom, which has 1 protons.
Contains H atom, which has 1 protons.


In [ ]:
# Now let's use chemical_series.py to make a hydrogen ring.
atom_spacing = 0.7414
basis = 'sto-3g'
atom_type = 'H'
biggest_ring = 20
print('\nAbout to make some hydrogen rings.')
for n_atoms in range(4, 20, 2):
    hydrogen_ring = make_atomic_ring(
        n_atoms, atom_spacing, basis, atom_type, autosave=autosave)
    print('Molecule has automatically generated name {}'.format(
        hydrogen_ring.name))
    print('This molecule has {} atoms and {} electrons.'.format(
        hydrogen_ring.n_atoms, hydrogen_ring.n_electrons))
    
# Now let's use chemical_series.py to make some atoms.
print('\nIt can be hard to guess the right spin multiplicity of some atoms. Let FermiLib do it:')
atomic_symbols = ['C', 'Na', 'Ca', 'Te', 'Cu', 'Fe', 'Ag']
for symbol in atomic_symbols:
    atom = make_atom(symbol, basis, autosave)
    print('{} has multiplicity {} and would be named {}.'.format(
        symbol, atom.multiplicity, atom.name))

## SparseOperator data structure stores matrices

Often, one would like to go a step further and directly map an operator or term to a sparse matrix which can be analyzed numerically. The sparse_operators.py module of our code provides very basic functionality to do this. In particular, the method .get_sparse_operator() can be called on any QubitTerm/Operator or FermionTerm/Operator to return an instance of our SparseOperator data structure. There are numerous methods one can call on the sparse operators such as "get_gap", "get_hartree_fock_state", "get_ground_state", ect. We show this off with the Hubbard model.

In [ ]:
from hubbard import fermi_hubbard

# Set model.
x_dimension = 2
y_dimension = 2
tunneling = 2.
coulomb = 1.
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 0

# Get qubit representation of Hubbard model.
hubbard_model = fermi_hubbard(
    x_dimension, y_dimension, tunneling, coulomb, 
    chemical_potential, magnetic_field, periodic, spinless)
qubit_hamiltonian = hubbard_model.jordan_wigner_transform()
print('Now printing the Jordan-Wigner representation of our Hubbard model:\n{}'.format(
    qubit_hamiltonian))

# Get sparse matrix representation of Hubbard model.
sparse_hamiltonian = qubit_hamiltonian.get_sparse_operator()
print('Some elements of the sparse Hamiltonian for this system are as follow:\n{}'.format(
    sparse_hamiltonian))

# Get the model gap and ground state energy.
energy, ground_state = sparse_hamiltonian.get_ground_state()
gap = sparse_hamiltonian.get_gap()
print('\nGround state energy of model is {} in units of t and J.'.format(energy))
print('The system gap is {} in units of t and J.'.format(gap))


In [ ]:
from molecular_data import MolecularData, _PERIODIC_TABLE
from chemical_series import make_atomic_ring, make_atom

# Set parameters to make a simple molecule.
diatomic_bond_length = 8.
geometry = [('Na', (0., 0., 0.)), ('Cl', (0., 0., diatomic_bond_length))]
basis = '6-31g'
multiplicity = 1
charge = 0
autosave = False
description = str(diatomic_bond_length)

# Make molecule and print out a few interesting facts about it.
molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description, autosave)
print('Molecule has automatically generated name {}'.format(
    molecule.name))
print('Information about this molecule would be saved at:\n{}\n'.format(
    molecule.data_handle()))
print('This molecule has {} atoms and {} electrons.'.format(
    molecule.n_atoms, molecule.n_electrons))
for atom, atomic_number in zip(molecule.atoms, molecule.protons):
    print('Contains {} atom, which has {} protons.'.format(
        atom, atomic_number))
    
# Now let's use chemical_series.py to make a hydrogen ring.
atom_spacing = 0.7414
basis = 'sto-3g'
atom_type = 'H'
biggest_ring = 20
print('\nAbout to make some hydrogen rings.')
for n_atoms in range(4, 20, 2):
    hydrogen_ring = make_atomic_ring(
        n_atoms, atom_spacing, basis, atom_type, autosave=autosave)
    print('Molecule has automatically generated name {}'.format(
        hydrogen_ring.name))
    print('This molecule has {} atoms and {} electrons.'.format(
        hydrogen_ring.n_atoms, hydrogen_ring.n_electrons))
    
# Now let's use chemical_series.py to make some atoms.
print('\nIt can be hard to guess the right spin multiplicity of some atoms. Let FermiLib do it:')
atomic_symbols = ['C', 'Na', 'Ca', 'Te', 'Cu', 'Fe', 'Ag']
for symbol in atomic_symbols:
    atom = make_atom(symbol, basis, autosave)
    print('{} has multiplicity {} and would be named {}.'.format(
        symbol, atom.multiplicity, atom.name))


## Running psi4 to populate the MolecularData class

The module run_psi4.py provides a user-friendly way of running psi4 calculations in FermiLib. The basic idea is that once one generates a MolecularData instance, one can then call psi4 with a specification of certain options (for instance, how much memory to use and what calculations to do) in order to compute things about the molecule, update the MolecularData object, and save results of the calculation. The most common calculations users will want in FermiLib would probably be self-consistent field (aka Hartree-Fock calculations). Our code uses these "SCF" calculations compute orbitals, integrals, Hartree-Fock energy, and more. Other common calculations are CISD and FCI calculations which also compute the 1-RDM and 2-RDM associated with their answers, CCSD calculations which also compute the CCSD amplitudes (useful for UCC) and MP2 calculations which can also be used to UCCSD initial guesses. 
Note that the "delete_input" and "delete_output" options indicate whether to save automatically generated psi4 input files or not.

It is because psi4 has a GPL license that the only code which interacts with psi4 is run_psi4.py. In order to avoid license contamination we actually use python code to generate an input file for psi4 using psi4_template. Python code in psi4_template instructs psi4 (not python) to load the MolecularData object, populate it with information from calculations and then save the MolecularData object as a pickle or (eventually HDF5). The module run_psi4 uses subprocess to actually run_psi4 and then loads the pickled MolecularData. Let us look at a simple example where we compute the energy of H$_2$ at various bond lengths.

Warnings: electronic structure calculations are finicky. They sometimes fail for surprising reasons. If a particular calculation is not converging it is probably necessary to change some of the SCF options set in psi4_template. See the Psi4 documentation for more information or consult and electronic structure theory expert.

In [47]:
from fermilib.utils import MolecularData
from psi4tmp import run_psi4

# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
bond_length_interval = 0.2
n_points = 10

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Generate molecule at different bond lengths.
hf_energies = []
fci_energies = []
bond_lengths = []
for point in range(1, n_points + 1):
    bond_length = bond_length_interval * float(point)
    bond_lengths += [bond_length]
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    molecule = MolecularData(
        geometry, basis, multiplicity,
        autosave=False, description=str(bond_length))
    
    # Run Psi4.
    molecule = run_psi4(molecule,
                        run_scf=run_scf,
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)

    # Print out some results of calculation.
    print('\nAt bond length of {} Bohr, molecular hydrogen has:'.format(
        bond_length))
    print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
    print('MP2 energy of {} Hartree.'.format(molecule.mp2_energy))
    print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
    print('Nuclear repulsion energy between protons is {} Hartree.'.format(
        molecule.nuclear_repulsion))
    for orbital in range(molecule.n_orbitals):
        print('Spatial orbital {} has energy of {} Hartree.'.format(
            orbital, molecule.orbital_energies[orbital]))
    hf_energies += [molecule.hf_energy]
    fci_energies += [molecule.fci_energy]

# Plot.
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(0)
plt.plot(bond_lengths, fci_energies, 'x-')
plt.plot(bond_lengths, hf_energies, 'o-')
plt.ylabel('Energy in Hartree')
plt.xlabel('Bond length in Bohr')
plt.show()

ModuleNotFoundError: No module named 'run_psi4'

## InteractionOperator and InteractionRDM for efficient numerical representations

Molecular Hamiltonians can be expressed as $H = h_0 + \sum_{pq} h_{pq}\, a^\dagger_p a_q + \frac{1}{2} \sum_{pqrs} h_{pqrs} \, a^\dagger_p a^\dagger_q a_r a_s$ where $h_0$ is a constant shift due to the nuclear repulsion and $h_{pq}$ and $h_{pqrs}$ are the famous molecular integrals. Since fermions interact pairwise, their energy is thus a unique function of the one-particle and two-particle reduced density matrices which are expressed in second quantization as $\rho_{pq} = \left \langle p \mid a^\dagger_p a_q \mid q \right \rangle$ and $\rho_{pqrs} = \left \langle pq \mid a^\dagger_p a_q \mid rs \right \rangle$, respectively.

Because both the RDMs and molecular Hamiltonians are both compactly represented and manipulated as 2- and 4- index tensors, we can represent them in a particularly efficient form using  similar data structures. The InteractionOperator data structure can be initialized for a Hamiltonian (or RDM) by passing the constant $h_0$ (or 0), as well as numpy arrays representing $h_{pq}$ (or $\rho_{pq}$) and $h_{pqrs}$ (or $\rho_{pqrs}$). Importantly, InteractionOperators can also be obtained by calling MolecularData.get_molecular_hamiltonian() or FermionOperator.get_molecular_operator(). The InteractionRDM data structure is similar but represents RDMs. For instance, one can get a molecular RDM by calling MolecularData.get_molecular_rdm(). When generating Hamiltonians from the MolecularData class, one can choose to restrict the system to an active space.

These classes both inherit from the same base class, InteractionTensor. This data structure overloads the slice operator [] so that one can get or set the key attributes of the InteractionOperator: .constant, .one_body_coefficients and .two_body_coefficients. For instance, InteractionOperator[p,q,r,s] would return $h_{pqrs}$ and InteractionRDM would return $\rho_{pqrs}$. It also supports binary operators !=, ==. Importantly, the class supports fast basis transformations using the method InteractionTensor.rotate_basis(rotation_matrix).

But perhaps most importantly, one can map the InteractionOperator to any of the other data structures we've described here. The generation of a QubitOperator representation of the InteractionOperator using the Jordan-Wigner transformation is shown below.

In [ ]:
from molecular_data import MolecularData
from run_psi4 import run_psi4
import scipy
import scipy.linalg
import numpy

# Set molecule parameters.
diatomic_bond_length = 4.
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
description = str(diatomic_bond_length)

# Set Hamiltonian parameters.
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData using Psi4.
molecule = MolecularData(geometry, basis, multiplicity,
                         description=description, autosave=False)
molecule = run_psi4(molecule, run_scf=True)

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    active_space_start, active_space_stop)
print('\nThe associated molecular Hamiltonian follows:\n{}\n'.format(
    molecular_hamiltonian))

# Map operator to fermions and qubits.
fermion_hamiltonian = molecular_hamiltonian.get_fermion_operator()
qubit_hamiltonian = molecular_hamiltonian.jordan_wigner_transform()
print('The associated qubit Hamiltonian follows:\n{}'.format(qubit_hamiltonian))
sparse_hamiltonian = qubit_hamiltonian.get_sparse_operator()
energy, state = sparse_hamiltonian.get_ground_state()
print('Ground state energy before rotation is {} Hartree.'.format(energy))

# Randomly rotate.
n_orbitals = molecular_hamiltonian.n_qubits // 2
n_variables = n_orbitals * (n_orbitals - 1) / 2
random_angles = numpy.pi * (1. - 2. * numpy.random.rand(n_variables))
kappa = numpy.zeros((n_orbitals, n_orbitals))
index = 0
for p in range(n_orbitals):
    for q in range(p + 1, n_orbitals):
        kappa[p, q] = random_angles[index]
        kappa[q, p] = -numpy.conjugate(random_angles[index])
        index += 1

    # Build the unitary rotation matrix.
    difference_matrix = kappa + kappa.transpose()
    assert numpy.amax(numpy.absolute(difference_matrix)) < 1e-9
    rotation_matrix = scipy.linalg.expm(kappa)

    # Apply the unitary.
    molecular_hamiltonian.rotate_basis(rotation_matrix)
qubit_hamiltonian = molecular_hamiltonian.jordan_wigner_transform()
sparse_hamiltonian = qubit_hamiltonian.get_sparse_operator()
energy, state = sparse_hamiltonian.get_ground_state()
print('Ground state energy after rotation is {} Hartree.'.format(energy))